In [3]:
from pybaseball import statcast
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Variables to consider to build a predicted hit model

- launch angle
- launch speed
- hc_y
- hc_x
- if_fielding_assignment (need to dummy code)
- of_fielding_assignment (need to dummy code)
- outs_when_up
- home_team (need to dummy code)
- stand (recode)
- hit_location
- p_throws (recode)

In [4]:
hitter_stats = statcast('2019-03-31','2019-10-04')

This is a large query, it may take a moment to complete
Completed sub-query from 2019-03-31 to 2019-04-05
Completed sub-query from 2019-04-06 to 2019-04-11
Completed sub-query from 2019-04-12 to 2019-04-17
Completed sub-query from 2019-04-18 to 2019-04-23
Completed sub-query from 2019-04-24 to 2019-04-29
Completed sub-query from 2019-04-30 to 2019-05-05
Completed sub-query from 2019-05-06 to 2019-05-11
Completed sub-query from 2019-05-12 to 2019-05-17
Completed sub-query from 2019-05-18 to 2019-05-23
Completed sub-query from 2019-05-24 to 2019-05-29
Completed sub-query from 2019-05-30 to 2019-06-04
Completed sub-query from 2019-06-05 to 2019-06-10
Completed sub-query from 2019-06-11 to 2019-06-16
Completed sub-query from 2019-06-17 to 2019-06-22
Completed sub-query from 2019-06-23 to 2019-06-28
Completed sub-query from 2019-06-29 to 2019-07-04
Completed sub-query from 2019-07-05 to 2019-07-10
Completed sub-query from 2019-07-11 to 2019-07-16
Completed sub-query from 2019-07-17 to 2019-

In [6]:
hit = hitter_stats[['launch_angle','launch_speed','hc_x','hc_y','if_fielding_alignment','of_fielding_alignment','outs_when_up','home_team',
    'stand','hit_location','p_throws','type','events']]

in_play = hit[hit['type']=='X']

## Recode events to hits or not

In [7]:
in_play['hit'] = in_play['events'].apply(lambda x: 1 if x in('single','double','home_run','triple') else 0)

## Cleaning Data for input into the model
- Recodes
- Dummy Codes

In [8]:
# Recodes
in_play['right_handed_batter'] = in_play['stand'].apply(lambda x: 1 if x =='R' else 0)
in_play['right_handed_pitcher'] = in_play['p_throws'].apply(lambda x: 1 if x =='R' else 0)

In [9]:
# drop all variables that will be recoded or dummy coded
in_play_new = in_play.drop(['if_fielding_alignment','of_fielding_alignment','home_team','stand','type','events','p_throws'],axis=1)

In [10]:
# dummy code and concatenate 
train = pd.concat([in_play_new, pd.get_dummies(in_play['if_fielding_alignment'])],axis=1)
train = pd.concat([in_play_new, pd.get_dummies(in_play['of_fielding_alignment'])],axis=1)
train = pd.concat([in_play_new, pd.get_dummies(in_play['home_team'])],axis=1)

In [11]:
# make sure there are no NAs
train.dropna(inplace=True)

In [12]:
x_train = train.drop(['hit'],axis=1)
y_train = train['hit']

In [18]:
y_train.mean()

0.29614954255249126

predicting all outs would get us to 79.4% accuracy

## Logistic Regression

In [13]:
# cross-validate basic Logistic Regression model to get a baseline AUC
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
lr_cv = cross_val_score(LogisticRegression(),
x_train,y_train,scoring='roc_auc',cv=10)
print("Random Forest cross-validated mean AUC: ",lr_cv.mean())

Random Forest cross-validated mean AUC:  0.8614609439675309


## Random Forest

In [14]:
# cross-validate basic Random Forest model to get an AUC
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
rf_cv = cross_val_score(RandomForestClassifier(),
x_train,y_train,scoring='roc_auc',cv=10)
print("Random Forest cross-validated mean AUC: ",rf_cv.mean())

Random Forest cross-validated mean AUC:  0.9515462122534686


## XGBoost

In [13]:
# cross-validate basic XGBoost model to get an AUC
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
xgb_cv = cross_val_score(XGBClassifier(),
x_train,y_train,scoring='roc_auc',cv=10)
print("XGBoost cross-validated mean AUC: ",xgb_cv.mean())

XGBoost cross-validated mean AUC:  0.9540392410573311


# Train/Test Split

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(x_train,y_train,test_size=.35, random_state=101)
print (len(X_train), len(X_test), len(X_train)+ len(X_test))

75965 40905 116870


In [16]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train,y_train)
xgb_proba = xgb.predict_proba(X_test)
# Predictions###############
xgb_proba_2 = xgb_proba[:,1]
xgb_pred = xgb.predict(X_test)

print('XGBoost Accuracy:',accuracy_score(y_test, xgb_pred))
print('XGBoost AUC:',roc_auc_score(y_test,xgb_proba_2))

XGBoost Accuracy: 0.9158782544921159
XGBoost AUC: 0.9550029834446239


In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_test,xgb_pred))

             precision    recall  f1-score   support

          0       0.92      0.96      0.94     28989
          1       0.90      0.80      0.85     11916

avg / total       0.92      0.92      0.91     40905



In 30 minutes I was able to produce a better model than the [hardballtimes model](https://tht.fangraphs.com/using-statcast-data-to-predict-hits/) on precision, recall, accuracy, and F1. 


Note model 2

![img](https://i.imgur.com/6aJvc7s.png)